    The dataset

In [2]:
from datasets import load_dataset
dataset = load_dataset("glue", "sst2")

In [3]:
print(dataset['train'][0])
print(dataset['train'].features)
print(dataset['train'].shape)
print(dataset['train'][0:5])

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}
{'sentence': Value('string'), 'label': ClassLabel(names=['negative', 'positive']), 'idx': Value('int32')}
(67349, 3)
{'sentence': ['hide new secretions from the parental units ', 'contains no wit , only labored gags ', 'that loves its characters and communicates something rather beautiful about human nature ', 'remains utterly satisfied to remain the same throughout ', 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up '], 'label': [0, 0, 1, 0, 0], 'idx': [0, 1, 2, 3, 4]}


Tokenizer

In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["sentence"], padding=True, truncation=True)

dataset_encoded = dataset.map(tokenize, batched=True)


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
print(dataset_encoded["train"].features)
print(dataset_encoded["train"].shape)
print(dataset_encoded["train"][0:1])

{'sentence': Value('string'), 'label': ClassLabel(names=['negative', 'positive']), 'idx': Value('int32'), 'input_ids': List(Value('int32')), 'token_type_ids': List(Value('int8')), 'attention_mask': List(Value('int8'))}
(67349, 6)
{'sentence': ['hide new secretions from the parental units '], 'label': [0], 'idx': [0], 'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}


Transformer le dataset en Tensor: Le modèle n’a pas besoin de la phrase texte ni de l’index — il apprend uniquement à relier input_ids → label.

In [6]:
dataset_encoded.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'label']
)

Le DataLoader est un outil PyTorch qui :

- Découpe ton dataset en petits lots (batches),

- Mélange les exemples pour éviter que le modèle apprenne par ordre fixe,

- Envoie automatiquement les données à chaque étape de ton entraînement.

In [7]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset_encoded["train"], batch_size=16, shuffle=True)